<a href="https://colab.research.google.com/github/Elsacred02/assignment_4_advanced_machine_learning/blob/main/Assignment4_parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras import layers

keras.utils.set_random_seed(42)

In [57]:
path = 'moby_dick.txt'
with open(path, encoding = 'utf-8') as f:
  text = f.read().lower()
print('Text length:', len(text))
print()
print(text[0 : 1000])

Text length: 1191786

chapter 1. loomings.

call me ishmael. some years ago—never mind how long precisely—having
little or no money in my purse, and nothing particular to interest me
on shore, i thought i would sail about a little and see the watery part
of the world. it is a way i have of driving off the spleen and
regulating the circulation. whenever i find myself growing grim about
the mouth; whenever it is a damp, drizzly november in my soul; whenever
i find myself involuntarily pausing before coffin warehouses, and
bringing up the rear of every funeral i meet; and especially whenever
my hypos get such an upper hand of me, that it requires a strong moral
principle to prevent me from deliberately stepping into the street, and
methodically knocking people’s hats off—then, i account it high time to
get to sea as soon as i can. this is my substitute for pistol and ball.
with a philosophical flourish cato throws himself upon his sword; i
quietly take to the ship. there is nothing surpri

In [58]:
chars = sorted(list(set(text)))

print('total chars:', len(chars))

#from chars to numbers
char_indices = dict((c, i) for i, c in enumerate(chars))
# from numbert to chars
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
print(indices_char)



total chars: 64
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, '£': 53, 'â': 54, 'æ': 55, 'è': 56, 'é': 57, 'œ': 58, '—': 59, '‘': 60, '’': 61, '“': 62, '”': 63}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: '(', 6: ')', 7: '*', 8: ',', 9: '-', 10: '.', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ';', 23: '?', 24: '[', 25: ']', 26: '_', 27: 'a', 28: 'b', 29: 'c', 30: 'd', 31: 'e', 32: 'f', 33: 'g', 34: 'h', 35: 'i', 36: 'j', 37: 'k', 38: 'l', 39: 'm', 40: 'n', 41: 'o', 42: 'p', 43: 'q', 44: 'r', 45: 's', 46: 't', 4

In [59]:
maxlen = 30
step = 2

sentences = []
next_chars = []

for i in range(0, 200000, step):
  sentences.append(text[i : i + maxlen])
  next_chars.append(text[i + maxlen])

print('number of sentences:', len(sentences))

number of sentences: 100000


In [60]:
x = np.zeros((len(sentences), maxlen, len(chars))) # 100,000 x 30 x 46
y = np.zeros((len(sentences), len(chars))) # 100,000 x 46

for i, (sentence, next_char) in enumerate(zip(sentences, next_chars)):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_char]] = 1

print(x.shape)
print(y.shape)

(100000, 30, 64)
(100000, 64)


In [61]:
model = keras.Sequential([
    keras.Input(shape = (30, len(chars), 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(len(chars), activation="softmax"),
])

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 28, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 12, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 14, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 5376)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 5376)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │       344,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,944 (1.38 MB)

 Trainable params: 362,944 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
from keras.callbacks import LambdaCallback
import sys

def testAfterEpoch(epoch, _):
  if epoch < 10:
    return
  print()
  print()
  print('----- Generating text after Epoch: %d' % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)

  generated = ''
  sentence = text[start_index : start_index + maxlen]
  generated += sentence
  print('**** starting sentence*****')
  print(sentence)
  print('***************************')
  sys.stdout.write(generated)

  for i in range(200):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1
    preds = model.predict(x_pred, verbose = 0)[0]
    next_index = np.argmax(preds)
    next_char = indices_char[next_index]

    sentence = sentence[1 : ] + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end = testAfterEpoch)



In [64]:
model.fit(x, y,
          batch_size = 2048,
          epochs = 30,
          callbacks = [print_callback])

KeyboardInterrupt: 